In [1]:
import os, re
os.chdir("/Users/Pace/Documents/Education/Universität Potsdam/Semester 4 - Summer 2024/Evaluating Chatp-Optimized Language Models/Project")
from backends import ModelSpec, Model, get_model_for, load_model_registry

### Game logic

In [2]:
class Game:
    def __init__(self, llm: Model, system_prompt: str, instructions: str) -> None:
        self.llm: Model = llm

        # Prompt and response attributes
        self.system_prompt: str = system_prompt
        self.instructions: str = instructions
        self.last_response: list = []
        self.context: list = []
        self.reprompt: bool = False
        self.tag: str = "MY MOVE:"
        self.sign: str = " -> "
        self.separator: str = " ; "

        # Constructs board with n fields
        self.n_fields: int = 23
        cells: list = []
        for _ in range(self.n_fields):
            cells.append("□")
        self.board = " ".join(cells).rstrip()
        self.current_state: str = self.board

        self.rolls: list = [6, 5, 4, 5, 6, 6, 2, 3, 6, 5, 1, 2, 5, 3, 4]
        self.turn: int = 0

        # Token attributes
        self.tokens_inplay: dict = {"X": False, "Y": False}
        self.current_position: dict = {"X": 0, "Y": 0}
        self.six_count: int = 0

    def make_move(self, reprompt_message: str | None = None) -> None:
        # Prepares the appropriate message for the current turn
        match self.turn:
            case 0:
                print(f"TURN 0: {self.board}")
                message: str = f"{self.instructions}\nAre you ready to play?\nBeginning state: {self.board}\nTurn number: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."

            case _:
                message: str = f"Current state: {self.current_state}\nNext turn number: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
                message = reprompt_message + message if reprompt_message else message

        # Delivers the message, then retrieves the response
        self._add_message(message)
        _, _, response_text = self.llm.generate_response(self.context)

        # Parses and saves the response
        move: dict = self._parse_reply(response_text)
        self.last_response = response_text

        # If the move is valid, updates everything accordingly
        if self._check_move(move, self.rolls[self.turn]):
            # print(self.current_state)
            self._add_message(response_text, role="assistant")
            for token in move.keys():
                self.tokens_inplay[token] = move[token] > 0
                self.current_position[token] = move[token]
            self._update_board(move)
            self.turn += 1
            print(f"TURN {self.turn}: {self.current_state}")

        # Otherwise, reports relevant information at the failing turn
        else:
            print(f"Fail at turn {self.turn}")
            print(f"Roll: {self.rolls[self.turn]}")
            print(f"Move: {move}")
            print(response_text)
            print(self.current_state)

            # if self.reprompt:
            #     self.make_move(self._reprompt(error_type="field number"))

    def _add_message(self, message: str, role: str = "user") -> None:
        if not self.context:
            self.context = [{"role": "system", "content": self.system_prompt}]
            
        self.context.append({"role": role, "content": message})

    def _parse_reply(self, reply: str) -> dict:
        matching_string: Match = re.search(r"MY MOVE: X -> (\d+) ; Y -> (\d+)", reply)

        if not matching_string:
            raise ValueError("Invalid response format")

        return {"X": int(matching_string.group(1)), "Y": int(matching_string.group(2))}

    def _check_token_moved(self, move: dict) -> dict:
        tokens_moved: dict = {}

        for token in move.keys():
            previous_location: int = self.current_position[token]
            current_location: int = move[token]
            tokens_moved[token] = previous_location != current_location

        return tokens_moved

    def _check_dual_token_moved(self, move: dict) -> bool:
        tokens_moved: list = []

        for token in move.keys():
            tokens_moved.append(self.current_position[token] != move[token])

        return True if all(tokens_moved) else False

    def _find_selected(self, tokens_moved: dict) -> str:
        for token in tokens_moved.keys():
            if tokens_moved[token]:
                selected: str = token

        return selected

    def _check_move(self, move: dict, roll: int) -> bool:
        state: list = []

        if self._check_dual_token_moved(move):
            print("MOVE ERROR: Both in-play tokens were moved simultaneously.")
            return False

        tokens_moved: dict = self._check_token_moved(move)
        selected: str = self._find_selected(tokens_moved)

        for token in move.keys():
            match [selected, self.tokens_inplay[token]]:
                
                # if the token was not moved and has not been played to the board
                case [False, False]: 
                    if roll != 6:
                        state.append(True)
                        continue

                    else:
                        print("MOVE ERROR: A 6 was rolled but the selected out-of-play token was not played to the board.")
                        return False
                
                # if the token was not moved but has been played to the board
                case [False, True]:
                    if roll + self.current_position[token] > self.n_fields:
                        state.append(True)
                        continue

                    else:
                        print("MOVE ERROR: The selected in-play token can be moved but was not.")
                        return False
                
                # if the selected token was moved
                case [True, _]: 
                    match [token == selected]:
                        
                        # if the selected token has been moved
                        case True: 
                            if roll == 6 and move[token] == 1:
                                state.append(True)
                                continue

                            elif self.current_position[token] + roll == move[token]:
                                state.append(True)
                                continue

                            else:
                                print("MOVE ERROR: The selected token was not moved appropriately.")
                                return False

                        # if the selected token has not been moved
                        case False: 
                            state.append(True)
                            continue
                            
        if all(state):
            return True

    def _reprompt(self, error_type: str) -> str:
        match error_type:
            case "field number":
                return f"There are 21 empty fields and 2 occupied fields. In total: 21 + 2 = 23 fields. Let's try again.\n"

    def _update_board(self, move: dict) -> None:
        split_board: list = self.board.split()
        
        for token in move.keys():
            position: int = move[token]
            if self.tokens_inplay[token]:
                split_board[self.current_position[token] - 1] = "□"
                split_board[position - 1] = token

        self.current_state = " ".join(split_board).rstrip()

    def _reset(self) -> None:
        self.turn = 0
        self.context = []

### Game Instantiation

In [3]:
# Prepares the LLM
load_model_registry()
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106")
llm: Model = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature=0.0, max_tokens=400)

# Loads the prompts
with open('games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    system_prompt: str = f.read()
with open('games/ludo/resources/prompts_pace/multitoken_v1.txt', 'r') as f:
    instructions: str = f.read()

# Defines the game instance
instance: Game = Game(llm, system_prompt, instructions)

In [4]:
for _ in range(10):
    instance.make_move()

TURN 0: □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
TURN 1: X □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
TURN 2: □ □ □ □ □ X □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
TURN 3: □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □ □ □ □ □ □
TURN 4: □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □
TURN 5: Y □ □ □ □ □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □
TURN 6: Y □ □ □ □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □ □
MOVE ERROR: Both in-play tokens were moved simultaneously.
Fail at turn 6
Roll: 2
Move: {'X': 13, 'Y': 3}
There are 22 empty fields and 2 occupied fields. In total: 22 + 2 = 24 fields. I have token X on field 13 and token Y on field 1. You have rolled 2. This is 2 fields away from my previous position. Since both tokens are on the field, I can try to move any of them. I decide to move my token Y. Since 1 + 2 = 3, I need to move my token to field number 3.
MY MOVE: X -> 13 ; Y -> 3
Y □ □ □ □ □ □ □ □ □ □ □ □ X □ □ □ □ □ □ □ □ □
2024-05-29 13:56:22,320 - backends.utils - WARNING - Found consecutive role assignments. T

ValueError: Invalid response format

In [ ]:
print(instance.last_response)

In [ ]:
import shutil
width, _ = shutil.get_terminal_size()

for turn_content in instance.context:
    print("=" * width + "\n")
    print(f"Role: {turn_content['role']}")
    print(turn_content["content"] + "\n\n")